# App en Dash


Thanks for your post.We are happy to assist you. If you are doing locally on  a jupyter notebook follow these steps.

Install first anaconda distribution using the link given 
below:  https://www.anaconda.com/products/individual  

As you scroll down, you will get the installers for Windows, Mac and Linux. 
Now, after the installation completes. Open Anaconda Navigator using run as administrator  

Click 'Launch' for Jupyter notebook. 

This will open the Jupyter notebook.in local host in browser. 

Now, you can create or open the Python 3 notebook.

Copy the skeleton code in

https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py


in the jupyter notebook.

Later include this code in top cell 

```!pip install dash==1.19.0  ```

```!pip install jupyter_dash ```

``` !pip install --upgrade plotly```


import pandas as pd  
import dash  
import dash_html_components as html  
import dash_core_components as dcc  
from dash.dependencies import Input, Output  
from jupyter_dash import JupyterDash  
import plotly.graph_objects as go


Further you need to change 

app=dash.Dash(__name__)

as

app = JupyterDash(__name__)

Change your last line

if __name__ == '__main__':
    app.run_server()

as

if __name__ == '__main__':
    app.run_server(mode='jupyterlab', port = 8090 ,dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)


This is the url of the spacex_launch_dash.csv

 dataset

https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv

In the spacex_dash_app.py replace the line

spacex_df = pd.read_csv("spacex_launch_dash.csv")

as

spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")


Make these changes execute your code

Later in browser use the url

http://localhost:8090

Whenever you make changes to your code and want to rerun it  close the running instance, shutdown the kernel and restart the kernel and rerun the cells.



Hope this helps

With Regards

Lakshmi Holla

## App en labs.cognitiveclass

In [ ]:
""""
# python3.11 spacex_dash_app.py
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# dropdown options
dropdown_options = [
    {'label': 'All Sites', 'value': 'ALL'},
    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
]

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=dropdown_options,
                                            value='ALL', #Default Vale
                                            placeholder="Select a Launch Site", #lo que aparece en la app
                                            searchable=True, #Funcionalidad de busqueda en lista desplegable
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    marks={i: f'{i} kg' for i in range(0, 10001, 1000)},
                                    value=[min_payload, max_payload]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
# Pie suma los valores numericos en cada celda de la columna, por eso me dara el mismo resultado en este caso
# si uso un df filtrado con los lanzamientos exitosos, o si uso todo el dataframe, ya que los ceros
# no van a sumar al tamano de cada seccion del pie. Si class tuviera otros valores, o fuera categorica o booleana, 
# ahi si seria necesario hacer un filtro y luego usar count() por ejemplo asi
#site_counts = filtered_df['Launch Site'].value_counts().reset_index()
#site_counts.columns = ['Launch Site', 'count']
def get_pie_chart(entered_site): 
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
            names='Launch Site', 
            title='Total Success Launches By Site')
        return fig
    else:
        condition = spacex_df['Launch Site']==entered_site
        filtered_df = spacex_df[condition]
        site_counts = filtered_df['class'].value_counts().reset_index()
        site_counts.columns = ['class', 'count']
        fig = px.pie(site_counts, values='count', 
            names='class', 
            title=f'Success vs. Failure Launches for {entered_site}')
        return fig


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
            Output(component_id='success-payload-scatter-chart', component_property='figure'),
            [Input(component_id='site-dropdown', component_property='value'), 
            Input(component_id="payload-slider", component_property="value")]
            )

def get_scatterplot(selected_site,payload_range): 
    low, high = payload_range
    # Filtrar el DataFrame según el rango de carga útil seleccionado
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]
    
    # Si se selecciona "ALL", mostramos todos los sitios
    if selected_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                         color='Booster Version Category',
                         title='Correlation between Payload and Success for All Sites')
    else:
        # Filtrar el DataFrame por el sitio de lanzamiento seleccionado
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                         color='Booster Version Category',
                         title=f'Correlation between Payload and Success for {selected_site}')
    
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(port=8090)

#esto es lo que se ejecuta en terminal
# python3.11 spacex_dash_app.py """

## App en Jupyter

In [1]:
# python3.11 spacex_dash_app.py esto es lo que toca escribir en terminal en labs.cognitives no se usa en Jupyter
# Import required libraries

!pip install -U dash
!pip install werkzeug==2.0.3


!pip install jupyter_dash
!pip install --upgrade plotly




import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash 
import plotly.express as px
#not sure whether the following line must be executed
import plotly.graph_objects as go

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = JupyterDash(__name__)

# dropdown options
dropdown_options = [
    {'label': 'All Sites', 'value': 'ALL'},
    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
]

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=dropdown_options,
                                            value='ALL', #Default Vale
                                            placeholder="Select a Launch Site", #lo que aparece en la app
                                            searchable=True, #Funcionalidad de busqueda en lista desplegable
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    marks={i: f'{i} kg' for i in range(0, 10001, 1000)},
                                    value=[min_payload, max_payload]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
# Pie suma los valores numericos en cada celda de la columna, por eso me dara el mismo resultado en este caso
# si uso un df filtrado con los lanzamientos exitosos, o si uso todo el dataframe, ya que los ceros
# no van a sumar al tamano de cada seccion del pie. Si class tuviera otros valores, o fuera categorica o booleana, 
# ahi si seria necesario hacer un filtro y luego usar count() por ejemplo asi
#site_counts = filtered_df['Launch Site'].value_counts().reset_index()
#site_counts.columns = ['Launch Site', 'count']
def get_pie_chart(entered_site): 
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
            names='Launch Site', 
            title='Total Success Launches By Site')
        return fig
    else:
        condition = spacex_df['Launch Site']==entered_site
        filtered_df = spacex_df[condition]
        site_counts = filtered_df['class'].value_counts().reset_index()
        site_counts.columns = ['class', 'count']
        fig = px.pie(site_counts, values='count', 
            names='class', 
            title=f'Success vs. Failure Launches for {entered_site}')
        return fig


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
            Output(component_id='success-payload-scatter-chart', component_property='figure'),
            [Input(component_id='site-dropdown', component_property='value'), 
            Input(component_id="payload-slider", component_property="value")]
            )

def get_scatterplot(selected_site,payload_range): 
    low, high = payload_range
    # Filtrar el DataFrame según el rango de carga útil seleccionado
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]
    
    # Si se selecciona "ALL", mostramos todos los sitios
    if selected_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                         color='Booster Version Category',
                         title='Correlation between Payload and Success for All Sites')
    else:
        # Filtrar el DataFrame por el sitio de lanzamiento seleccionado
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                         color='Booster Version Category',
                         title=f'Correlation between Payload and Success for {selected_site}')
    
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(mode='jupyterlab', port = 8090 ,dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)

#esto es lo que se ejecuta en terminal
# python3.11 spacex_dash_app.py

#URL de navegador para la app
# http://localhost:8090

   ---------------------------------------- 0.0/7.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/7.5 MB 1.6 MB/s eta 0:00:05
   - -------------------------------------- 0.3/7.5 MB 3.2 MB/s eta 0:00:03
   ------------ --------------------------- 2.3/7.5 MB 16.0 MB/s eta 0:00:01
   --------------------------- ------------ 5.2/7.5 MB 27.8 MB/s eta 0:00:01
   ---------------------------------------- 7.5/7.5 MB 31.8 MB/s eta 0:00:00
  Attempting uninstall: dash-table
    Found existing installation: dash-table 4.11.2
    Uninstalling dash-table-4.11.2:
      Successfully uninstalled dash-table-4.11.2
  Attempting uninstall: dash-html-components
    Found existing installation: dash-html-components 1.1.2
    Uninstalling dash-html-components-1.1.2:
      Successfully uninstalled dash-html-components-1.1.2
  Attempting uninstall: dash-core-components
    Found existing installation: dash-core-components 1.15.0
    Uninstalling dash-core-components-1.15.0:
      Successfull

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 2.2.5 requires Werkzeug>=2.2.2, but you have werkzeug 2.0.3 which is incompatible.


  Using cached werkzeug-3.0.3-py3-none-any.whl.metadata (3.7 kB)
Using cached werkzeug-3.0.3-py3-none-any.whl (227 kB)
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.0.3
    Uninstalling Werkzeug-2.0.3:
      Successfully uninstalled Werkzeug-2.0.3


C:\Users\faibe\AppData\Local\Temp\ipykernel_12356\1103871188.py:16: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\faibe\AppData\Local\Temp\ipykernel_12356\1103871188.py:17: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\faibe\anaconda3\Lib\site-packages\dash\dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [ ]:
1 + 1